In [5]:
import os
import luigi
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
PROJECT_DIR = os.path.join(os.getcwd(), os.pardir)
os.chdir(PROJECT_DIR)

In [8]:
from src.features.dtm import CreateDTM

In [9]:
PROJECT_DIR

'C:\\Users\\wertu\\Documents\\Datascience\\udacity-ml-capstone\\notebooks\\..'

In [10]:
luigi.build([CreateDTM()], local_scheduler = True)

DEBUG: Checking if CreateDTM() is complete
INFO: Informed scheduler that task   CreateDTM__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=140461088, workers=1, host=DESKTOP-6UJS098, username=wertu, pid=2468) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 present dependencies were encountered:
    - 1 CreateDTM()

Did not run any tasks
This progress looks :) because there were no failed tasks or missing external dependencies

===== Luigi Execution Summary =====



True

In [11]:
features = joblib.load("data/processed/dtm_features.pkl")

In [12]:
train = joblib.load("data/interim/train.pkl")
test = joblib.load("data/interim/test.pkl")

# Random Forest

In [9]:
rf = RandomForestClassifier(n_estimators = 50, max_depth = 500, n_jobs=5)

In [10]:
rf.fit(features["train"], train.funny)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=500, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=5,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
rf.score(features["train"], train.funny)

0.99220151688142233

In [12]:
rf.score(features["test"], test.funny)

0.67140002854288572

In [13]:
check = rf.feature_importances_

In [14]:
check

array([  3.57385008e-03,   1.52317068e-04,   4.77038963e-04, ...,
         3.73054524e-07,   1.89129137e-07,   6.49569501e-07])

In [15]:
import numpy as np

In [16]:
indexes = np.argpartition(check, 100)[-100:]

In [17]:
indexes

array([  1821,   1812,   1751,   1733,   1690,   1687,   1653, 145591,
         1597, 145593,   1594,   1540, 145596,   1537,   1492, 145599,
         1462,   1457,   1436,   1434,   1420,   1409,   1395,   1384,
       145608,   1381,   1377,   1282,   1242,   1227,   1201,   1200,
         1161,   1160,   1158, 145619,   1134,   1120,   1110,   1101,
         1077, 145625,   1076,   1072,   1064,   1055,   1053,    992,
          970,    967,    949,    925,    857,    843,    797,    796,
          768,    765,    760,    742,    723,    722,    714,    713,
          681,    648,    647,    635,    633,    626,    620,    557,
          543,    505, 145658,    483,    430,    405,    396,    389,
          324,    312,    230,    217,    204, 145669,    201,    197,
          196,    172,    143,     89,     87,     78,      6,      5,
            4,      3,      2,      0], dtype=int64)

In [18]:
np.array(features["vectorizer"].get_feature_names())[indexes]

array(["'T", "'Sure", "'So", "'Sir", "'See", "'Second", "'S", '😀', "'RE",
       '😁', "'R", "'Please", '😂', "'Plagiarism", "'P", '😂😂', "'Okay",
       "'Oh", "'Now", "'Not", "'No", "'Netflix", "'NSFW", "'N", '😃', "'My",
       "'Murica", "'M", "'Lee", "'LL", "'K", "'Just", "'Its", "'It", "'Is",
       '😉', "'If", "'I", "'How", "'Honey", "'Hi", '😎', "'Hey", "'Here",
       "'Hello", "'He", "'Have", "'Good", "'Get", "'Genius'", "'G",
       "'Forget", "'Excuse", "'Ernie", "'E", "'Dwarf", "'Doctor", "'Do",
       "'Did", "'Dear", "'Daddy", "'Dad", "'D", "'Cuz", "'Come",
       "'Caution-", "'Cause", "'Can", "'Ca", "'C", "'But", "'Big",
       "'Because", "'B", '😭', "'Are", "'After", "'A", "'90s", "'80s", "'1",
       "'-axis", "''You", "''What", "''Ugh", '🙄', "''This", "''The",
       "''That", "''Shut", "''Oh", "''I", "''How", "''He", "''", "'", '&',
       '%', '$', '!'],
      dtype='<U700')

# Boosted Trees

In [2]:
test = []
test.append(2)
test.append(3)
test

[2, 3]

In [189]:
class EarlyStopper():
    def __init__(self, validation_set, validation_labels, patience = 0):
        self.validation_set = validation_set
        self.validation_labels = validation_labels
        self.accuracy = []
        self.loss = [np.inf]
        self.preds = None
        self.probs = None
        self.patience = patience
        self.lowest_loss = np.inf
        
    def __call__(self, i, estimator, fit_stage_locals):
        
        if i == 0:
            self.preds = estimator.staged_predict(self.validation_set)
            self.probs = estimator.staged_predict_proba(self.validation_set)
            
        preds = next(self.preds)
        probs = next(self.probs)
        
        acc = np.mean(preds == self.validation_labels )
        loss = log_loss(self.validation_labels, probs)
        
        self.accuracy.append(acc)
        self.loss.append(loss)
        
        print("Iteration: {}".format(i))
        print("Loss: {}".format(loss))
        print("Accuracy: {}".format(acc))
        
        if (min(self.loss[max((i-self.patience),0):i+1]) > self.lowest_loss) & (i > self.patience):
            return True
        else:
            self.lowest_loss = 
            return False
        

In [190]:
import numpy as np

In [191]:
gmb_stopper = EarlyStopper(features["test"], test.funny.values.astype(np.int64), patience = 5)

In [192]:
gbm = GradientBoostingClassifier(n_estimators=500)

In [193]:
gbm.fit(features["train"], train.funny.values.astype(np.int64), monitor = gmb_stopper)

Iteration: 0
Loss: 0.6875785372713787
Accuracy: 0.5815256172399029
Iteration: 1
Loss: 0.6833037130572504
Accuracy: 0.590124161552733
Iteration: 2
Loss: 0.6798110287852657
Accuracy: 0.5942985585842728
Iteration: 3
Loss: 0.6769375297967754
Accuracy: 0.59433423719138
Iteration: 4
Loss: 0.6742320839506454
Accuracy: 0.5950121307264165
Iteration: 5
Loss: 0.6720937133634003
Accuracy: 0.5952618809761667
Iteration: 6
Loss: 0.6700759942667311
Accuracy: 0.5979020979020979
Iteration: 7
Loss: 0.6684481777278891
Accuracy: 0.5996860282574569
Iteration: 8
Loss: 0.6670271098443951
Accuracy: 0.6001498501498501
Iteration: 9
Loss: 0.665998646859855
Accuracy: 0.6000428143285286
Iteration: 10
Loss: 0.6649313061300225
Accuracy: 0.6022905665762809
Iteration: 11
Loss: 0.6639003049591506
Accuracy: 0.6026830312544599
Iteration: 12
Loss: 0.6631447026236789
Accuracy: 0.6018624232909947
Iteration: 13
Loss: 0.6624199684721305
Accuracy: 0.6022548879691737
Iteration: 14
Loss: 0.6616344316083818
Accuracy: 0.60261167404

Iteration: 121
Loss: 0.6461332269954168
Accuracy: 0.6194876552019409
Iteration: 122
Loss: 0.6461033917668759
Accuracy: 0.6192379049521907
Iteration: 123
Loss: 0.6460176910859113
Accuracy: 0.619309262166405
Iteration: 124
Loss: 0.6459686759592962
Accuracy: 0.6195233338090481
Iteration: 125
Loss: 0.645934480445295
Accuracy: 0.6191665477379763
Iteration: 126
Loss: 0.6458560445468351
Accuracy: 0.6196303696303697
Iteration: 127
Loss: 0.6457644560062383
Accuracy: 0.6197730840587984
Iteration: 128
Loss: 0.6457166598677557
Accuracy: 0.6201655487369773
Iteration: 129
Loss: 0.6456307528615051
Accuracy: 0.6200585129156557
Iteration: 130
Loss: 0.6456045040465609
Accuracy: 0.6200228343085485
Iteration: 131
Loss: 0.64554302024918
Accuracy: 0.6200941915227629
Iteration: 132
Loss: 0.6454916417399418
Accuracy: 0.6201298701298701
Iteration: 133
Loss: 0.6454449460587773
Accuracy: 0.6201655487369773
Iteration: 134
Loss: 0.6454073478236979
Accuracy: 0.6203439417725132
Iteration: 135
Loss: 0.645350156168383

Iteration: 241
Loss: 0.641351989744757
Accuracy: 0.6280148423005566
Iteration: 242
Loss: 0.6413341980515591
Accuracy: 0.6281575567289853
Iteration: 243
Loss: 0.6413390342576697
Accuracy: 0.6281575567289853
Iteration: 244
Loss: 0.6413116749951998
Accuracy: 0.628300271157414
Iteration: 245
Loss: 0.6412671413117368
Accuracy: 0.6283359497645212
Iteration: 246
Loss: 0.6412354711395437
Accuracy: 0.6283716283716284
Iteration: 247
Loss: 0.6412169738283755
Accuracy: 0.6283716283716284
Iteration: 248
Loss: 0.6411977258351729
Accuracy: 0.6285857000142715
Iteration: 249
Loss: 0.6411471725207983
Accuracy: 0.6285857000142715
Iteration: 250
Loss: 0.6411463087316128
Accuracy: 0.6286213786213786
Iteration: 251
Loss: 0.6411301832017622
Accuracy: 0.6284073069787356
Iteration: 252
Loss: 0.6411047380175936
Accuracy: 0.6286213786213786
Iteration: 253
Loss: 0.6411056147455382
Accuracy: 0.6286213786213786
Iteration: 254
Loss: 0.6410844823601293
Accuracy: 0.6290852005137719
Iteration: 255
Loss: 0.6410455890003

Iteration: 361
Loss: 0.638122503615329
Accuracy: 0.6336163836163836
Iteration: 362
Loss: 0.6381029688104286
Accuracy: 0.6335450264021693
Iteration: 363
Loss: 0.6380587673419066
Accuracy: 0.6339018124732411
Iteration: 364
Loss: 0.6380505891350079
Accuracy: 0.6339731696874554
Iteration: 365
Loss: 0.6380711651147037
Accuracy: 0.6340088482945626


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [141]:
max(np.inf, 0.0)

inf

In [167]:
check = [np.inf, 2, -9]

In [173]:
min(check[11:-3:-1])

-9